## Game

In [49]:
import pygame
import sys
import random
import time


# Initialisierung
pygame.init()
CLOCK = pygame.time.Clock()
SCREEN = pygame.display.set_mode((1280, 720))
pygame.display.set_caption("YOLOv5 Game")

# Startposition des Spielers
X_POSITION, Y_POSITION = 50, 440

# Lade Schriftart
GAME_FONT = pygame.font.Font("assets/MotleyForcesRegular-w1rZ3.ttf", 60)

# Spielgeschwindigkeit
GAME_SPEED = 5

PLAYER_SCORE = 0

# Spieler-Klasse
class Player(pygame.sprite.Sprite):
    def __init__(self, x_pos, y_pos):
        super().__init__()

        # Animationen für das Laufen, Ducken und Springen
        self.running_sprites = [
            pygame.transform.scale(pygame.image.load("assets/Dino1.png"), (80, 100)),
            pygame.transform.scale(pygame.image.load("assets/Dino2.png"), (80, 100))
        ]
        self.ducking_sprites = [
            pygame.transform.scale(pygame.image.load("assets/DinoDucking1.png"), (110, 60)),
            pygame.transform.scale(pygame.image.load("assets/DinoDucking2.png"), (110, 60))
        ]
        self.jumping_surface = pygame.transform.scale(pygame.image.load("assets/DinoJumping.png"), (80, 100))

        self.x = x_pos
        self.y = y_pos

        self.current_image = 0
        self.image = self.running_sprites[self.current_image]
        self.rect = self.image.get_rect(center=(self.x, self.y))

        self.velocity = 200
        self.gravity = 1.15
        self.ducking = False
        self.is_jumping = False

    def update(self):
        self.animate()
        self.apply_gravity()
        self.move()

    def animate(self):
        self.current_image += 0.05
        if self.current_image >= 2:
            self.current_image = 0

        if self.ducking:
            self.image = self.ducking_sprites[int(self.current_image)]
        elif self.is_jumping:
            self.image = self.jumping_surface
        else:
            self.image = self.running_sprites[int(self.current_image)]

    def duck(self):
        self.ducking = True
        self.rect.centery = 460 

    def unduck(self):
        self.ducking = False
        self.rect.centery = 440

    def apply_gravity(self):
        if self.rect.centery <= 440:
            self.rect.centery += self.gravity

    def jump(self):
        if self.rect.centery >= 440:
            self.velocity = -30
            self.is_jumping = True

    def move(self):
        if self.is_jumping:
            self.rect.centery += self.velocity
            self.velocity += self.gravity
            if self.rect.centery >= 440:
                self.rect.centery = 440
                self.velocity = 0
                self.is_jumping = False


class JumpObject(pygame.sprite.Sprite):
    def __init__(self, x_pos, y_pos):
        super().__init__()
        self.x_pos = x_pos
        self.y_pos = y_pos
        self.sprites = []
        for i in range(1, 6):
            current_sprite = pygame.transform.scale(
                pygame.image.load(f"assets/cacti/cactus{i}.png"), (100, 100))
            self.sprites.append(current_sprite)
        self.image = random.choice(self.sprites)
        self.rect = self.image.get_rect(center=(self.x_pos, self.y_pos))

    def update(self):
        self.x_pos -= GAME_SPEED  # Verwenden Sie GAME_SPEED aus Ihrem vorhandenen Code
        self.rect = self.image.get_rect(center=(self.x_pos, self.y_pos))



PAUSE_COLOR = (100, 100, 100, 200)
PAUSE_FONT_COLOR = (255, 255, 255)
paused = False

def draw_pause_menu():
    # Erstellen einer leicht transparenten dunkelgrauen Schicht
    pause_surface = pygame.Surface(SCREEN.get_size(), pygame.SRCALPHA)
    pause_surface.fill(PAUSE_COLOR)
    SCREEN.blit(pause_surface, (0, 0))

    # Zeichnen des Hintergrunds und des Spielers (optional)
    SCREEN.blit(ground, (ground_x, 0))
    SCREEN.blit(ground, (ground_x + 1280, 0))
    player_group.draw(SCREEN)

    pause_text = GAME_FONT.render("PAUSE", True, PAUSE_FONT_COLOR)
    pause_rect = pause_text.get_rect(center=(SCREEN.get_width() // 2, SCREEN.get_height() // 2 - 40))
    SCREEN.blit(pause_text, pause_rect)

    pygame.display.update()


def end_game():
    global GAME_SPEED, PLAYER_SCORE, game_over, ground_x

    game_over = True

    while game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    # Hier setzen Sie die Spielvariablen zurück
                    PLAYER_SCORE = 0
                    GAME_SPEED = 5
                    obstacle_group.empty()
                    game_over = False
                    ground_x = 0  # Setzen Sie die Position des Bodens zurück
                    player.rect.centery = Y_POSITION  # Setzen Sie die Spielerposition zurück
                    jump_objects_group.empty()  # Löschen Sie alle JumpingObjects

        # Zeichnen Sie den aktuellen Bildschirm ohne das Spiel zu aktualisieren
        SCREEN.blit(ground, (ground_x, 0))
        SCREEN.blit(ground, (ground_x + 1280, 0))
        player_group.draw(SCREEN)
        jump_objects_group.draw(SCREEN)  # Zeichnen Sie die JumpingObjects im Game Over-Fenster
        obstacle_group.draw(SCREEN)

        # Zeichnen Sie das Game Over und den Score in Weiß
        game_over_text = GAME_FONT.render("Game Over!", True, (255, 255, 255))
        game_over_rect = game_over_text.get_rect(center=(640, 250))
        score_text = GAME_FONT.render(f"Score: {int(PLAYER_SCORE)}", True, (255, 255, 255))
        score_rect = score_text.get_rect(center=(640, 310))
        SCREEN.blit(game_over_text, game_over_rect)
        SCREEN.blit(score_text, score_rect)

        # Text für die Neustart-Anweisung
        restart_text = GAME_FONT.render("Press SPACE to restart", True, (255, 255, 255))
        restart_rect = restart_text.get_rect(center=(640, 370))
        SCREEN.blit(restart_text, restart_rect)

        pygame.display.update()






# Hintergrund und Spielergruppe
ground = pygame.image.load("assets/background.png")
ground = pygame.transform.scale(ground, (1280, 900))
ground_x = 0
ground_rect = ground.get_rect(center=(640, 400))

player_group = pygame.sprite.GroupSingle()
player = Player(X_POSITION, Y_POSITION)
player_group.add(player)

jump_objects_group = pygame.sprite.Group()

obstacle_group = pygame.sprite.Group()


next_jump_object_time = time.time() + random.uniform(0.5, 2)  # Erster Zeitpunkt für die Erstellung


# Kollisionsdetektion
if pygame.sprite.spritecollide(player, jump_objects_group, False):
    game_over = True


# Hauptspiel-Schleife
while True:
    keys = pygame.key.get_pressed()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                paused = not paused  # Wechseln Sie den Pause-Zustand
            if event.key == pygame.K_SPACE or event.key == pygame.K_UP:
                player.jump()

    if not paused:
        if keys[pygame.K_DOWN]:
            player.duck()
        else:
            if player.ducking:
                player.unduck()

        player.update()

        GAME_SPEED += 0.0025
        PLAYER_SCORE += 0.1
        ground_x -= 1

        SCREEN.blit(ground, (ground_x, 0))
        SCREEN.blit(ground, (ground_x + 1280, 0))
        player_group.draw(SCREEN)

        if ground_x <= -1280:
            ground_x = 0

        
        current_time = time.time()
        if current_time >= next_jump_object_time:
            jump_object = JumpObject(1280, 440)  # Sie können die Y-Position nach Bedarf ändern
            jump_objects_group.add(jump_object)

            # Setzen Sie den nächsten Zeitpunkt für die Erstellung zufällig zwischen 0,5 und 2 Sekunden in die Zukunft
            next_jump_object_time = current_time + random.uniform(0.5, 2.0)

        jump_objects_group.update()
        jump_objects_group.draw(SCREEN)

        
        # Erstellen Sie den Text für den Punktestand
        score_text = GAME_FONT.render(f"Score: {int(PLAYER_SCORE)}", True, (255, 255, 255))

        # Positionieren Sie den Text oben rechts
        score_rect = score_text.get_rect(topright=(1200, 40))

        # Zeichnen Sie den Text auf den Bildschirm
        SCREEN.blit(score_text, score_rect)

        # Kollisionsdetektion mit JumpObject-Instanzen
        if pygame.sprite.spritecollide(player, jump_objects_group, False, pygame.sprite.collide_rect):
            game_over = True
            end_game()  # Rufen Sie die end_game-Funktion auf, wenn eine Kollision auftritt




    if paused:
        draw_pause_menu()

    pygame.display.update()
    CLOCK.tick(120)



SystemExit: 